# Healthy and Bleached Corals Image Classification using MobileNet

In [13]:
!pip install transformers torch torchvision torchaudio


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\krain\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

A custom Dataset class is created to load the coral images from the file paths HEALTHY_IMAGES_DIR and BLEACHED_IMAGES_DIR.
The subfolders contain images of healthy and bleached corals.

The dataset used is from Kaggle: https://www.kaggle.com/datasets/vencerlanz09/healthy-and-bleached-corals-image-classification/data

Dataset Details:
+ Total images: 923
+ Image categories: 2
    + Healthy corals: 438 images
    + Bleached corals: 485 images
+ Image format: JPEG
+ Image size: Maximum 300 px for either width or height, whichever is higher

After loading the images the dta is splitted into training and testing sets.

In [15]:
class CoralDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)

In [16]:
HEALTHY_IMAGES_DIR = "/data/healthy_corals"
healthy_image_paths = [os.path.join(HEALTHY_IMAGES_DIR, img) for img in os.listdir(HEALTHY_IMAGES_DIR) if os.path.isfile(os.path.join(HEALTHY_IMAGES_DIR, img))]
BLEACHED_IMAGES_DIR = "/data/bleached_corals"
bleached_image_paths = [os.path.join(BLEACHED_IMAGES_DIR, img) for img in os.listdir(BLEACHED_IMAGES_DIR) if os.path.isfile(os.path.join(BLEACHED_IMAGES_DIR, img))]

image_paths = healthy_image_paths + bleached_image_paths
labels = [0] * len(healthy_image_paths) + [1] * len(bleached_image_paths)

# Split the data into train and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

In [17]:
# Modify batch size, learning rate, augmentation, and epochs dynamically

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

runs = [
    {"batch_size": 40, "lr": 0.0001, "augmentation": "default", "epochs": 10},
    {"batch_size": 60, "lr": 0.00005, "augmentation": "horizontal_flip", "epochs": 15},
    {"batch_size": 20, "lr": 0.0002, "augmentation": "rotation", "epochs": 10},
    {"batch_size": 40, "lr": 0.0001, "augmentation": "color_jitter", "epochs": 20},
]

# Load the pretrained MobileNet model
model = AutoModelForImageClassification.from_pretrained("google/mobilenet_v2_1.0_224")

# Replace the classifier layer to output a single value for binary classification
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, 1),
    nn.Sigmoid()
)

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set up the optimizer and loss function
criterion = nn.BCELoss()

for i, run in enumerate(runs):
    print(f"\n### Running Configuration {i + 1} ###\n")
    
    # Set data augmentation
    if run["augmentation"] == "default":
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])
    elif run["augmentation"] == "horizontal_flip":
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])
    elif run["augmentation"] == "rotation":
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])
    elif run["augmentation"] == "color_jitter":
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])
    
    # Update dataset and dataloaders
    train_dataset = CoralDataset(train_paths, train_labels, transform=transform)
    val_dataset = CoralDataset(val_paths, val_labels, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=run["batch_size"], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=run["batch_size"], shuffle=False)
    
    # Update optimizer learning rate
    optimizer = optim.Adam(model.parameters(), lr=run["lr"])
    
    # Train the model
    for epoch in range(run["epochs"]):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images).logits.squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images).logits.squeeze()
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{run['epochs']}], Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%")


### Running Configuration 1 ###

Epoch [1/10], Loss: 0.5849, Val Loss: 0.5252, Val Accuracy: 75.68%


KeyboardInterrupt: 



### **Result Table After Each Run**

| **Run** | **Batch Size** | **Learning Rate** | **Augmentation**                | **Epochs** | **Final Train Loss** | **Final Val Loss** | **Final Val Accuracy** |
|---------|----------------|-------------------|----------------------------------|------------|-----------------------|---------------------|-------------------------|
| 1       | 40             | 0.0001           | Resize, Normalize               | 10         |                       |                     |                         |
| 2       | 60             | 0.00005          | Horizontal Flip                 | 15         |                       |                     |                         |
| 3       | 20             | 0.0002           | Rotation (15°)                  | 10         |                       |                     |                         |
| 4       | 40             | 0.0001           | Horizontal Flip + Color Jitter  | 20         |                       |                     |                         |
